Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

# Summarization using T5 model from Hugging Face

### Summarization with T5-3B model
We will use the Hugging Face Summariazion example with the T5-3B model to fine tune the model with the CNN-dailymail dataset

run_summarization.py is a lightweight example of how to download and preprocess a dataset from the 🤗 Datasets library 

#### Initial Setup
we start with a Habana PyTorch Docker image and run this notebook

#### Install Habana's DeepSpeed Fork
Habana's DeepSpeed Fork has implementations specifically for Gaudi and must be used

In [1]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.10.0  

  Cloning https://github.com/HabanaAI/DeepSpeed.git (to revision 1.10.0) to /tmp/pip-req-build-01wrf_8x
  Running command git clone --filter=blob:none --quiet https://github.com/HabanaAI/DeepSpeed.git /tmp/pip-req-build-01wrf_8x
  Running command git checkout -b 1.10.0 --track origin/1.10.0
  Switched to a new branch '1.10.0'
  Branch '1.10.0' set up to track remote branch '1.10.0' from 'origin'.
  Resolved https://github.com/HabanaAI/DeepSpeed.git to commit 141faf783dac331bc3852590d05190dc0e883e51
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for deepspeed: filename=deepspeed-0.7.7+hpu.synapse.v1.10.0-py3-none-any.whl size=738066 sha256=250ee12193373e0448d3b264ebe09a5b0cd11fbbf6e93ac472ff253b97617831
  Stored in directory: /tmp/pip-ephem-wheel-cache-c6u4mmu2/wheels/a1/1d/50/40a4b5d2dcc3710881387fe9c3712cfbc73

#### Install the Optimum Habana Library

In [2]:
!python -m pip install optimum[habana]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 11.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 266.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 15.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 22.0 MB/s eta 0:00:00a 0:00:01
  

#### Clone the Hugging Face Model Repository

In [3]:
!git clone  https://github.com/huggingface/optimum-habana

Cloning into 'optimum-habana'...
remote: Enumerating objects: 4001, done.
remote: Counting objects: 100% (1112/1112), done.
remote: Compressing objects: 100% (400/400), done.
remote: Total 4001 (delta 768), reused 938 (delta 679), pack-reused 2889
Receiving objects: 100% (4001/4001), 2.13 MiB | 5.21 MiB/s, done.
Resolving deltas: 100% (2550/2550), done.


#### Go the Summarization example model and install the requirements

In [4]:
%cd optimum-habana/examples/summarization

/root/LLM-Training/optimum-habana/examples/summarization


In [5]:
!pip install -q -r requirements.txt

### Setup for DeepSpeed
Since we are using DeepSpeed, we have to confirm that the model has been configured properly.  We look for the following:

* deepspeed.initialize(model, ...) model, optimizer, ... = deepspeed.initialize(args=args, model=model, optimizer=optimizer, ...)
* deepspeed.init_distributed(dist_backend=“hccl”, init_method=init_method)
* Create a ds_config.json file to set the DS training parameters
  
  


#### DeepSpeed Initialization
Look in deepspeed.py and we see the model being passed to the DeepSpeed engine

```
    import deepspeed
    from deepspeed.utils import logger as ds_logger

    model = trainer.model
    args = trainer.a
    ...

    kwargs = {
        "args": habana_args,
        "model": model,
        "model_parameters": model_parameters,
        "config_params": config,
        "optimizer": optimizer,
        "lr_scheduler": lr_scheduler,
    }

    deepspeedengine, optimizer, _, lr_scheduler = deepspeed.initialize(**kwargs)rgs

```

#### DeepSpeed Distrbuted
Look in training_args.py and we see the DeepSpeed Distribution initialization

```
    from habana_frameworks.torch.distributed.hccl import initialize_distributed_hpu
    world_size, rank, self.local_rank = initialize_distributed_hpu()

    import deepspeed
    deepspeed.init_distributed(dist_backend="hccl", timeout=timedelta(seconds=self.ddp_timeout))
       logger.info("DeepSpeed is enabled.")
```

#### Create DeepSpeed Config file with ZeRO preferences
The ds_config.json file will configure the parameters to run DeepSpeed

In this case, we will run the ZeRO3 optimier and BF16 mixed precision.

In [19]:
%cd /root/LLM-Training/optimum-habana/examples/summarization

/root/LLM-Training/optimum-habana/examples/summarization


In [20]:
%%sh
tee ./ds_config.json <<EOF
{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 3,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false,
        "stage3_gather_16bit_weights_on_model_save": true
    }
}
EOF

{
    "steps_per_print": 64,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "bf16": {
        "enabled": true
    },
    "gradient_clipping": 1.0,
    "zero_optimization": {
        "stage": 3,
        "overlap_comm": false,
        "reduce_scatter": false,
        "contiguous_gradients": false,
        "stage3_gather_16bit_weights_on_model_save": true
    }
}


#### Fine Tuning T5-3b with the cnn_dailymail dataset
The T5-3b model is a large language model that was origianlly trained on the C4 dataset and in this case will be fined tuned on the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset that is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail.

For use of this model on First-Gen Gaudi, users should update the model to "T5-large"

This is run by `gaudi_spawn.py`, a simple launcher script to collect arguments and send them to `distributed_runner.py` for training on multiple HPUs, which then calls the `run_summarization.py` model.

Notice the Habana specific commands to use here:

-- use_habana  - allows training to run on Habana Gaudi  
-- use_hpu_graphs - reduces recompilation by replaying the graph  
-- gaudi_config_name Habana/t5 - mapping to Hugging Face T5 Model  

**Even though a Billion parameter T5 model can be used for Fine Tuning, this fine tuning still takes many hours to complete.  
For users that wish to execute the example Fine Tuning, they should modify the `model_name_or_path` to "t5-small", which only takes a few hours to complete.**


In [23]:
%pwd

'/root/LLM-Training/optimum-habana/examples/summarization'

In [12]:
!mkdir ft-summarization

mkdir: cannot create directory ‘ft-summarization’: File exists


**Run the command below in a terminal window** from the optimum-habana/examples/summarization folder, this is most likely here: 
/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization/

```
python ../gaudi_spawn.py \
--world_size 8 --use_deepspeed run_summarization.py \
--model_name_or_path t5-3b \
--do_train \
--do_eval \
--dataset_name cnn_dailymail \
--dataset_config '"3.0.0"' \
--source_prefix '"summarize: "' \
--output_dir ./ft-summarization \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size 4 \
--overwrite_output_dir \
--predict_with_generate \
--use_habana \
--use_lazy_mode \
--use_hpu_graphs \
--gaudi_config_name Habana/t5 \
--ignore_pad_token_for_loss False \
--pad_to_max_length \
--save_strategy epoch \
--throughput_warmup_steps 3 \
--deepspeed ./ds_config.json
```


### After fine tuning, let's look at the results
This fine tuned model has created the new `pytorch_model.bin` and the global_step.. folder contains the checkpoints that will be used in the infernece in the next section.


In [25]:
%cd ./ft-summarization

/root/LLM-Training/optimum-habana/examples/summarization/ft-summarization


In [26]:
%ls -al

total 217716
drwxr-xr-x 6 root root      4096 Jun 14 18:44 ./
drwxr-xr-x 5 root root      4096 Jun 14 06:58 ../
-rw-r--r-- 1 root root       312 Jun 14 18:44 all_results.json
drwxr-xr-x 3 root root      4096 Jun 14 17:48 checkpoint-17946/
drwxr-xr-x 3 root root      4096 Jun 14 18:44 checkpoint-26919/
drwxr-xr-x 3 root root      4096 Jun 14 16:53 checkpoint-8973/
-rw-r--r-- 1 root root      1474 Jun 14 18:44 config.json
-rw-r--r-- 1 root root       506 Jun 14 18:44 gaudi_config.json
-rw-r--r-- 1 root root       142 Jun 14 18:44 generation_config.json
-rw-r--r-- 1 root root 219639653 Jun 14 18:44 pytorch_model.bin
drwxr-xr-x 3 root root      4096 Jun 14 15:56 runs/
-rw-r--r-- 1 root root      2201 Jun 14 18:44 special_tokens_map.json
-rw-r--r-- 1 root root    791656 Jun 14 18:44 spiece.model
-rw-r--r-- 1 root root      2324 Jun 14 18:44 tokenizer_config.json
-rw-r--r-- 1 root root   2422095 Jun 14 18:44 tokenizer.json
-rw-r--r-- 1 root root     13541 Jun 14 18:44 trainer_state.json
-rw-

#### Summarization using the Pipeline
Now we can run the summarization using Hugging Face Pipeline call with the fine tuned model.  In this case we will point to the model that we fine tuned.   Remember that if you used t5-small to do the Fine Tuning, be sure to change the `model_to_finetune` to "t5-small"

In [31]:
import torch
import habana_frameworks.torch

from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load model to fine-tune and its tokenizer
model_to_finetune = "t5-3b"
model = AutoModelForSeq2SeqLM.from_pretrained(model_to_finetune)
tokenizer = AutoTokenizer.from_pretrained(model_to_finetune)

# Point to the ft-summarization folder with the fine-tuned model
path_to_local_model = "/root/Gaudi2-Workshop/LLM-Training/optimum-habana/examples/summarization/ft-summarization"

# Instantiate pipeline from local repo, if you did not run the fine tuning step above, you can change: model=model_to_finetune
pipe = pipeline(task="summarization", model=path_to_local_model, device="hpu", torch_dtype=torch.bfloat16)

text_to_summarize = "summarize: Introduction: The Strategic Arms Limitation Talks II (SALT II) treaty, signed on June 18, 1979, between the United States and the Soviet Union, marked a significant milestone in nuclear arms control efforts during the Cold War era. Building upon its predecessor, SALT I, the treaty aimed to curb the arms race and reduce the risk of nuclear conflict between the superpowers. Key Provisions: SALT II encompassed several crucial provisions. It placed limits on strategic offensive arms, including intercontinental ballistic missiles (ICBMs), submarine-launched ballistic missiles (SLBMs), and heavy bombers. The agreement specified the maximum number of deployed warheads and launchers each party could possess. Verification and Compliance: To ensure compliance, the treaty established comprehensive verification measures. This involved regular exchanges of data, on-site inspections, and monitoring activities by both nations. These measures sought to enhance transparency, foster trust, and prevent either side from gaining a significant advantage in terms of strategic nuclear capabilities. Ratification and Challenges: Although both the United States and the Soviet Union signed the treaty, its ratification faced considerable challenges. The political landscape changed when the Soviet Union invaded Afghanistan in 1979, leading to a deterioration of U.S.-Soviet relations. As a result, the United States never ratified the treaty formally, rendering it non-binding. However, both nations pledged to adhere to its principles, effectively implementing its provisions on a voluntary basis. Legacy and Impact: Despite the treaty's non-ratification, SALT II's legacy and impact were significant. It set the stage for subsequent arms control negotiations, providing a framework for future agreements such as the Intermediate-Range Nuclear Forces (INF) Treaty and the Strategic Arms Reduction Treaty (START). SALT II demonstrated the potential for cooperation between the superpowers and laid the groundwork for continued dialogue aimed at reducing the nuclear threat globally."
print("------------------------------------------------------------")
print("Input:", text_to_summarize)
print()

result = pipe(text_to_summarize)
print("------------------------------------------------------------")
print("Result:", result)



------------------------------------------------------------
Input: summarize: Introduction: The Strategic Arms Limitation Talks II (SALT II) treaty, signed on June 18, 1979, between the United States and the Soviet Union, marked a significant milestone in nuclear arms control efforts during the Cold War era. Building upon its predecessor, SALT I, the treaty aimed to curb the arms race and reduce the risk of nuclear conflict between the superpowers. Key Provisions: SALT II encompassed several crucial provisions. It placed limits on strategic offensive arms, including intercontinental ballistic missiles (ICBMs), submarine-launched ballistic missiles (SLBMs), and heavy bombers. The agreement specified the maximum number of deployed warheads and launchers each party could possess. Verification and Compliance: To ensure compliance, the treaty established comprehensive verification measures. This involved regular exchanges of data, on-site inspections, and monitoring activities by both nati